In [ ]:
# if we want to give our custom model to someone else then we'll be giving the directory my_ssd_mobnet 
# which conatains pipeline and checkpoints 
# we're going to load the latest checkpoints located inside my_ssd_mobnet directory

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PIPELINE_CONFIG = os.path.join('TensorFlow', 'Workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config')
CHECKPOINT_PATH = os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME) 

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-9')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

#### Detect from an Image

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
LABEL_MAP_NAME = 'label_map.pbtxt'
ANNOTATION_PATH = os.path.join('TensorFlow', 'Workspace','annotations')
LABELMAP = os.path.join(ANNOTATION_PATH, LABEL_MAP_NAME)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP)

In [ ]:
IMAGE_PATH = os.path.join('TensorFlow', 'Workspace','Images')
# 2021-06-13-084543
# borrowed_from_train
IMAGE_PATH = os.path.join(IMAGE_PATH, 'test', '2021-06-13-084543.jpg')
print(IMAGE_PATH)

#### Real Time Detections from your Webcam¶


In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

#### Freezing the graph

In [ ]:
# It is equivalent of saving your model

In [ ]:
APIMODEL_PATH = os.path.join('TensorFlow','models')

In [ ]:
FREEZE_SCRIPT = os.path.join(APIMODEL_PATH, 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
OUTPUT_PATH = os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME, 'export')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,PIPELINE_CONFIG, CHECKPOINT_PATH, OUTPUT_PATH)

In [ ]:
print(command)

#### Performance tuning

In [ ]:
# Adding more images of low performing classes to your dataset 
# Training for longer
# Changine architecture  # using another model from model zoo
